In [33]:
!pip install torch_geometric --user 
!pip install torch_sparse --user 
!pip install torch_scatter --user 
!pip install pytorch_lightning --user 
!jupyter labextension install @jupyter-widgets/jupyterlab-manager

/home/ai/.pyenv/versions/3.7.9/lib/python3.7/site-packages/jupyter_server/transutils.py:13: FutureWarning: The alias `_()` will be deprecated. Use `_i18n()` instead.
  warnings.warn(warn_msg, FutureWarning)
An error occured.
PermissionError: [Errno 13] Permission denied: '/home/ai/.pyenv/versions/3.7.9/share/jupyter/lab/extensions/jupyter-widgets-jupyterlab-manager-3.0.0.tgz'
See the log file for details:  /tmp/jupyterlab-debug-wfxbt06_.log


# Raw Data Loading Function 

In [34]:
import pandas as pd

def load_raw_cities_df():
    cities_df = pd.DataFrame({
        'city':['bangalore','Mumbai','Delhi','kolkatta','chennai','bhopal'],
        'lat':[12.9716,19.076,28.7041,22.5726,13.0827,23.2599],
        'lon':[77.5946,72.877,77.1025,88.639,80.2707,77.4126],
        'x1':[20,35,24,33,35,18],
        'x2':[5,5,7,13,16,21],
        'y':[1200,1500,2000,1780,1450,3000]})
    return cities_df 


# Preprocessing Functions 

In [35]:
import pandas as pd
import numpy as np 
from sklearn.neighbors import DistanceMetric
import networkx as nx
import torch 
from torch_geometric.utils.convert import from_networkx
from torch_geometric.data import Data

def convert_lat_lon_to_radian(cities_df):
    cities_df['lat'] = np.radians(cities_df['lat'])
    cities_df['lon'] = np.radians(cities_df['lon'])
    return cities_df
def get_city_dist_matrix(cities_df):
    dist = DistanceMetric.get_metric('haversine')
    cities_df[['lat','lon']].to_numpy()
    dist_matrix_df = pd.DataFrame(dist.pairwise(cities_df[['lat','lon']].to_numpy())*6373,  columns=cities_df.city.unique(), index=cities_df.city.unique())
    return dist_matrix_df

def convert_to_networkx_graph(dist_matrix_df, threshold = 1200):
    D = dist_matrix_df.values
    D[D < threshold] = 1
    D[D >= threshold] = 0
    G = nx.from_numpy_matrix(D)
    return G

def convert_to_geometric_data(x_df, y_df, G):
    x = torch.tensor(x_df.values, dtype=torch.float)
    y = torch.tensor(y_df.values, dtype=torch.float)
    edge_index = from_networkx(G).edge_index
    data = Data(x=x, y=y, edge_index=edge_index)
    return data

# Model Modules

In [36]:
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
import torch.nn.functional as F

class GraphNeuralNet(torch.nn.Module):
    def __init__(self, num_node_features, hidden_dim=2, output_dim=1):
        super(GraphNeuralNet, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        # x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return x
    
class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr='add')
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # Step 1: Add self-loops
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Multiply with weights
        x = self.lin(x)

        # Step 3: Calculate the normalization
        row, col = edge_index
        deg = degree(row, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # Step 4: Propagate the embeddings to the next layer
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x,
                              norm=norm)

    def message(self, x_j, norm):
        # Normalize node features.
        return norm.view(-1, 1) * x_j

# Define Lightning Module 

In [38]:
import pytorch_lightning as pl 

class AVMProjModule(pl.LightningModule):
    def __init__(self, num_node_features, hidden_dim = 2, output_dim=1, lr = 0.001):
        super(AVMProjModule, self).__init__()
        self.model = GraphNeuralNet(num_node_features, hidden_dim = hidden_dim, output_dim = output_dim)
        self.lr = lr 
    def forward(self, x, edge_index):
        y = self.model(x, edge_index)
        return y 
    def training_step(self, batch, batch_idx):
        x, y, edge_index = batch.x, batch.y, batch.edge_index 
        y_hat = self(x, edge_index)
        loss = F.l1_loss(y_hat, y)
        return {'loss': loss}
    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.lr) 

# Connect them all 

In [39]:
from get_geometric_data import get_geometric_data
from avm_modules import AVMProjModule

from torch_geometric.data import DataLoader 
geometric_data = get_geometric_data()


train_loader = DataLoader([geometric_data], batch_size=1)


project_module = AVMProjModule(
    geometric_data.num_node_features,
    hidden_dim=2, 
    output_dim=1
)

# from pytorch_lightning.loggers import TensorBoardLogger
# logger = TensorBoardLogger('/home/ai/work/logs/tensorboard', 'GNN_Refactor_Project')
trainer = pl.Trainer(auto_lr_find=True)
# logger.experiment.add_graph(project_module, geometric_data)

from tqdm.auto import tqdm
trainer.fit(project_module, train_loader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name  | Type           | Params
-----------------------------------------
0 | model | GraphNeuralNet | 9     
-----------------------------------------
9         Trainable params
0         Non-trainable params
9         Total params
0.000     Total estimated model params size (MB)
